## Request and Inspect Weekly SPX-Options

First, load ib-ruby environment

In [1]:
require_relative '../lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

Check the Presence of predefined indices

In [2]:
Symbols::Index.all

[:a_d, :asx, :dax, :hsi, :minihsi, :spx, :stoxx, :tick, :trin, :vasx, :vdax, :vhsi, :vix, :volume, :vstoxx]

In [ ]:
> Symbols::Index holds a selection of predefined indices

In [3]:
spx_market_price = Symbols::Index.spx.market_price

TWS Error 354: Requested market data is not subscribed.Delayed market data is available.Error&CBOE/IND/Top&CBOE/IND/Top
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.


4219.52

_Note:_ If the request is fired before the market opening, the returned market price is the closing price of the previous day.

 * Get the most recent expiry 
 * Define a range of strikes to consider as  market-price +/- 15 points.
 


In [4]:
# declare variables to be modified in the loop
spxw_options=  nil
date =  DateTime.now

loop do  
 spxw_options =  Option.new( trading_class: 'SPXW', 
                             symbol: 'SPX', 
                             currency: 'USD', 
                             expiry: date.strftime("%Y%m%d").to_i,
                             exchange: 'SMART', 
                             right: :put ).verify do |c|
      c if c.strike > spx_market_price - 15 && c.strike < spx_market_price + 15 
   end
 if spxw_options.empty?
   date = date.tomorrow
 else
   break
 end
end
  
puts spxw_options.as_table

TWS Error 200: No security definition has been found for the request
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ SPX    │ 481094285 │  SMART   │ 20210611 │    100     │     SPXW      │  put  │ 4210.0 │   USD    │
│ Option │ SPX    │ 481094294 │  SMART   │ 20210611 │    100     │     SPXW      │  put  │ 4220.0 │   USD    │
│ Option │ SPX    │ 481094297 │  SMART   │ 20210611 │    100     │     SPXW      │  put  │ 4225.0 │   USD    │
│ Option │ SPX    │ 481094300 │  SMART   │ 20210611 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 484266933 │  SMART   │ 20210611 │    100     │     SPXW      │  put  │ 4205.0 │   USD    │
│ Option │ SPX    │ 484266938 │  SMART   │ 

Lets inspect these Options further.

The method _request_greeks_ requests Tickdata from the TWS-Server and collects the calculated greeks 


In [5]:
spxw_options.map{|c| c.request_greeks thread: true }.each &:join
puts spxw_options.greek.as_table

TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent

### Get ATM-Weeklies

There is a method `Contract#atm-options`. Its designed for monthly options

In [6]:
atm_options =  Symbols::Index.spx.atm_options

atm_options.keys    # Expiries of the Options

TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Error 200: No security definition has been found for the request
TWS Error 200: No security definition has been found for the request
TWS Error 200: No security definition has been found for the request
TWS Error 200: No security definition has been found for the request
TWS Error 200: No security definition has been found for the request
TWS Error 200: No security definition has been found for the request
TWS Error 200: No security definition has been found for the request
TWS Error 200: No security definition has been found for the request
TWS Error 200: No security definition has been found for the request


[2106, 2107, 2108, 2109, 2111, 2112, 2201, 2202, 2203, 2205, 2206, 2212, 2301]

In [10]:
puts atm_options[2106].as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ SPX    │ 467200275 │  SMART   │ 20210617 │    100     │      SPX      │  put  │ 4220.0 │   USD    │
└────────┴────────┴───────────┴──────────┴──────────┴────────────┴───────────────┴───────┴────────┴──────────┘


To get all available ATM-SPXW-Options we define a sample (as above) and specify the strike

In [8]:
spxw =  Option.new trading_class: :spxw, symbol: :spx, currency: :usd, strike: 4230, 
                   exchange: :smart, right: :put

spxw_chain =  spxw.verify.sort_by &:last_trading_day
puts spxw_chain.as_table


┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ SPX    │ 490487010 │  SMART   │ 20210609 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 481094300 │  SMART   │ 20210611 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 495222203 │  SMART   │ 20210614 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 491690724 │  SMART   │ 20210616 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 467183869 │  SMART   │ 20210618 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 492917794 │  SMART   │ 20210621 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│

In [12]:
spxw_chain.map{|o| o.request_greeks( thread: true)}.each &:join
puts spxw_chain.greek.as_table

TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TO

TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
┌────────────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                         │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞════════════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ put  SPX/SPXW 20210609  4230.0 │      -- │         -- │       -- │      -- │      -- │      -- │      -- │
│ put  SPX/SPXW 20210611  4230.0 │   24.68 │       0.13 │     1.36 │   -0.68 │    0.01 │    0.95 │      -- │
│ put  SPX/SPXW 20210614  4230.0 │   29.07 │       0.10 │     1.52 │   -0.64 │    0.01 │    1.74 │   -1.98 │
│ put  SPX/SPXW 20210616  4230.0 │   36.15 │       0.12 │     1.82 │   -0.60 │    0.01 │    2.16 │   -1.95 │
│ put  SPX/SPXW 20210618  4230.0 │   42.17 │ 